In [1]:
# Importar las librerias
import pandas as pd

In [3]:
# Cargar los datos como DataFrame
spotify_2023 = pd.read_csv('spotify-2023.csv', encoding='ISO-8859-1')
spotify_2023

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


In [4]:
# Observar las columnas presentes en el set de datos
spotify_2023.columns

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%'],
      dtype='object')

In [5]:
spotify_2023['released_year'].unique()

array([2023, 2019, 2022, 2013, 2014, 2018, 2017, 2020, 2016, 2012, 1999,
       2008, 1975, 2021, 2015, 2011, 2004, 1985, 2007, 2002, 2010, 1983,
       1992, 1968, 1984, 2000, 1997, 1995, 2003, 1973, 1930, 1994, 1958,
       1957, 1963, 1959, 1970, 1971, 1952, 1946, 1979, 1950, 1942, 1986,
       2005, 1991, 1996, 1998, 1982, 1987], dtype=int64)

Tomando en cuenta las columnas presentes en el set de datos, se pueden definir las columnas de la siguiente forma:

- **track_name:** *nombre de la canción*
- **artist(s)_name:** *nombre del o los artistas de la canción*
- **artist_count:** *número de artistas que contribuyen a la canción*
- **released_year:** *año en el cuál se lanzó la canción*
- **released_month:** *mes en el cual se lanzó la canción*
- **released_day:** *día en el cual se lanzó la canción*
- **in_spotify_playlist** *número de listas de reproducción donde se incluye la canción en Spotify*
- **in_spotify_charts:** *precencia y rango de la canción en las listas de éxito de Spotify*
- **streams:** *número total de reproducciones en Spotify*
- **in_apple_playlists:** *número de listas de reproducción donde se incluye la canción en Apple Music*
- **in_apple_charts:** *precencia y rango de la canción en las listas de éxito de Apple Music*
- **in_deezer_playlists:** *número de listas de reproducción donde se incluye la canción en Dreezer*
- **in_deezer_charts:** *precencia y rango de la canción en las listas de éxito de Apple Music*
- **in_shazam_charts:** *presencia y rango de la canción en las listas de éxito de shazam*
- **bpm:** *pulsaciones por minuto, una medida de tiempo de la canción*
- **key:** *tono de la canción*
- **mode:** *modo de la canción, (mayor o menor)*
- **danceability_%:** *porcentaje que indica que tan acta es la canción para bailar*
- **valence_%:** *positividad del contenido de la canción*
- **energy_%:** *percepción del nivel de energía de la canción*
- **acousticness_%:** *cantidad de sonidos acústicos de la canción*
- **instrumentalness_%:** *cantidad de contenido instrumental de la canción*
- **liveness_%:** *presencia de elementos de actuación en vivo*
- **speechiness_%:** *cantidad de palabras habladas en la canción*